In [1]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("dark")
plt.rcParams['figure.figsize'] = 16, 12
import pandas as pd
from tqdm import tqdm_notebook
import io
from PIL import Image
from glob import glob
from collections import defaultdict
import os
import pickle

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

from dbxfinder_lib.utils import ImageList, ImageListExFiles

In [2]:
out_dir = '/home/mephistopheies/storage2/data/dbxfinder/amazon/features/'

with open(os.path.join(out_dir, 'files.tsv'), 'r') as f:
    files = f.readlines()
files = [s.strip() for s in files]
files_inv = dict([(s, ix) for (ix, s) in enumerate(files)])

In [3]:
to_tensor = transforms.ToTensor()
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

scale_size = 224
def scace_min_size(img_pil, scale_size=224):
    img_pil = img_pil.copy()
    if img_pil.size[1] <= img_pil.size[0]:
        new_h = scale_size
        new_w = int(np.round(new_h * img_pil.size[0]/img_pil.size[1]))
        img_pil = img_pil.resize((new_w, new_h))
    if img_pil.size[1] > img_pil.size[0]:
        new_w = scale_size
        new_h = int(np.round(new_w * img_pil.size[1]/img_pil.size[0]))
        img_pil = img_pil.resize((new_w, new_h))
    return img_pil

    
ds = ImageListExFiles(
    os.path.join(out_dir, 'files.tsv'),
    transform=transforms.Compose([
        transforms.Lambda(lambda img: scace_min_size(img, scale_size)),
        transforms.CenterCrop(scale_size),
        to_tensor,
        normalize
    ]))

In [4]:
workers = 1
batch_size_val = 10

data_loader = torch.utils.data.DataLoader(
    ds,
    batch_size=batch_size_val, 
    shuffle=False,
    num_workers=workers, 
    pin_memory=True)

In [3]:
model = models.resnet152(pretrained=True)

# hooks = []
# avg_pool = nn.AdaptiveAvgPool2d(1)
# max_pool = nn.AdaptiveMaxPool2d(1)

# def extract_features_original(self, input, output):
#     self.last_values = output.cpu().data.numpy().squeeze()

# def extract_features_avg(self, input, output):
#     self.last_values_avg = avg_pool(output).cpu().data.numpy().squeeze()
    
# def extract_features_max(self, input, output):
#     self.last_values_max = max_pool(output).cpu().data.numpy().squeeze()
    
# hooks.append(model.avgpool.register_forward_hook(extract_features_avg))
# hooks.append(model.avgpool.register_forward_hook(extract_features_max))

# model = model.eval().cuda()

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /home/mephistopheies/.torch/models/resnet152-b121ed2d.pth
100%|██████████| 241530880/241530880 [01:02<00:00, 3860174.52it/s]


In [7]:
for X, _, files_batch in data_loader:
    X_var = Variable(X.cuda(), volatile=True)
    Z = model(X_var)
    
    
    break

In [8]:
model.avgpool.last_values_max.shape

(10, 2048)